In [5]:
from sklearn.decomposition import KernelPCA,PCA
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import f1_score,accuracy_score
from sklearn.model_selection import cross_validate
from pylmnn import LargeMarginNearestNeighbor as LMNN
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import image
import seaborn as sns
import xlwt

from sklearn.feature_selection import VarianceThreshold,SelectKBest,mutual_info_classif

In [2]:
# parameter X: data array of shape(n_sample,n_feature)
# parameter y: labels of shape(n_samples,)
# parameter kernel: kernerl used for each layer (here is fixed for arccosine)
# parameter classifier: classifier used for the output layer
# parameter arc_order: the order of activation function and arccosine kernel
# return the trained MKM model

# classified by KNN with cv

def MKM(X,y,l,kernel):
    New_X,New_y = X,y
    print(New_X.shape)
    KPCA_list = []
    FS_list = []
    w_list = []
    for i in range(l):
        size = New_X.shape[1]
        # K-layer
        KPCA = KernelPCA(n_components = size-80, kernel = kernel)
        K_new_X,K_new_y = KPCA.fit_transform(New_X),New_y
#         pca = PCA(n_components = size-80)
#         K_new_X,K_new_y = pca.fit_transform(New_X),New_y
        # Bookkeeping the KPCA model used in each layer
        KPCA_list.append(KPCA)
#         KPCA_list.append(pca)
        # P-layer
#         w = cv_w(K_new_X,K_new_y,range(1,4),range(10,30,5))
#         w_list.append(w)
#         P_new_X,P_new_y,dic_kept = mutual_info_FS(K_new_X,K_new_y,size-80-w)
#         P_new_X,P_new_y,dic_kept = mutual_info_FS(K_new_X,K_new_y,size-100)
#         FS_list.append(dic_kept)
#         New_X,New_y = P_new_X,P_new_y
        New_X,New_y = K_new_X,K_new_y
        print('Train Layer {} Done!'.format(i+1))
        print('Data has the shape {}'.format(New_X.shape))
    final_X, final_y =  New_X, New_y
    lmnn = LMNN(n_neighbors=20, max_iter=100, n_components= final_X.shape[1])
    lmnn.fit(final_X, final_y)
    knn = KNN(n_neighbors = 50)
    model = knn.fit(lmnn.transform(final_X), final_y)
#     model = classifier.fit(final_X, final_y)
    print('Model has been successfully trained!')
    print('----------------------------------------------------------------------')
    return model,KPCA_list,FS_list,w_list

In [3]:
# parameter X: data array of shape(n_sample,n_feature)
# parameter y: labels of shape(n_samples,)
# parameter data_test_X: data array of shape(n_sample,n_feature)
# parameter data_test_y: labels of shape(n_samples,)
# parameter l: the number of layers of kernel machines
# parameter MKM: the trained MKM model
# parameter kernel: kernerl used for each layer (here is fixed for arccosine)
# parameter arc_order: the order of activation function and arccosine kernel
# return accuracy score for test set

# classified by KNN with cv

def MKM__test(X,y,l,KPCA_list,FS_list,w_list,MKM):
    New_X,New_y = X,y
    for i in range(l):
        size = New_X.shape[1]
        dic_feature = {}
        for f in range(size):
            dic_feature[f] = New_X[:,f]
        # K-layer
        K_new_X,K_new_y = KPCA_list[i].transform(New_X),New_y
        # P-layer
#         w = cv_w(K_new_X,K_new_y,range(1,15),range(10,300))
#         P_new_X,P_new_y = mutual_info_FS(K_new_X,K_new_y,w)
#         for key in FS_list[i].keys():
#             FS_list[i][key] = dic_feature[key]
#         temp = np.array(list(FS_list[i].values())).T[:,:size-80-w_list[i]]
#         temp = np.array(list(FS_list[i].values())).T[:,:size-100]
#         P_new_X,P_new_y = temp,K_new_y
#         New_X,New_y = P_new_X,P_new_y
        New_X,New_y = K_new_X,K_new_y
        print('Test Layer {} Done!'.format(i+1))
        print('Data has the shape {}'.format(New_X.shape))
    final_X, final_y =  New_X, New_y
    predict_y = MKM.predict(final_X)
    print(accuracy_score(final_y,predict_y))
    print('Test is finished!')
    print('----------------------------------------------------------------------')
    return accuracy_score(final_y,predict_y)

In [4]:
accu_score_1 = []
# number of layers
for i in range(6):
    print('layer({})'.format(i))
    # datasets test to ensure robust
    accu_score_temp = []
    for d in range(5):
        # load and sample the training and test dataset
        data = pd.read_csv('mnist_test.csv',header = None).iloc[:5000,:].sample(n=3000)
        data_X = data.iloc[:,1:].to_numpy()
        data_y = data.iloc[:,0].to_numpy()
        data_test = pd.read_csv('mnist_test.csv',header = None).iloc[5000:,:].sample(n=1000)
        data_test_X = data_test.iloc[:,1:].to_numpy()
        data_test_y = data_test.iloc[:,0].to_numpy()
        
        model, KPCA_list ,FS_list,w_list = MKM(data_X,data_y,i,kernel = 'rbf')
        accu_score_temp.append(MKM__test(data_test_X,data_test_y,i,
                                        KPCA_list = KPCA_list,FS_list = FS_list,
                                        w_list = w_list,MKM = model))
    accu_score_1.append(np.round(sum(accu_score_temp)/len(accu_score_temp),3))
    print('----------------------------------------------------------------')

layer(0)
(3000, 784)
Model has been successfully trained!
----------------------------------------------------------------------
0.126
Test is finished!
----------------------------------------------------------------------
(3000, 784)
Model has been successfully trained!
----------------------------------------------------------------------
0.205
Test is finished!
----------------------------------------------------------------------
(3000, 784)
Model has been successfully trained!
----------------------------------------------------------------------
0.123
Test is finished!
----------------------------------------------------------------------
(3000, 784)
Model has been successfully trained!
----------------------------------------------------------------------
0.117
Test is finished!
----------------------------------------------------------------------
(3000, 784)
Model has been successfully trained!
----------------------------------------------------------------------
0.105
Test 

(3000, 784)
Train Layer 1 Done!
Data has the shape (3000, 704)
Train Layer 2 Done!
Data has the shape (3000, 624)
Train Layer 3 Done!
Data has the shape (3000, 544)
Train Layer 4 Done!
Data has the shape (3000, 464)
Model has been successfully trained!
----------------------------------------------------------------------
Test Layer 1 Done!
Data has the shape (1000, 704)
Test Layer 2 Done!
Data has the shape (1000, 624)
Test Layer 3 Done!
Data has the shape (1000, 544)
Test Layer 4 Done!
Data has the shape (1000, 464)
0.089
Test is finished!
----------------------------------------------------------------------
(3000, 784)
Train Layer 1 Done!
Data has the shape (3000, 704)
Train Layer 2 Done!
Data has the shape (3000, 624)
Train Layer 3 Done!
Data has the shape (3000, 544)
Train Layer 4 Done!
Data has the shape (3000, 464)
Model has been successfully trained!
----------------------------------------------------------------------
Test Layer 1 Done!
Data has the shape (1000, 704)
Test La

In [8]:
workbook = xlwt.Workbook()  # 新建一个工作簿
sheet = workbook.add_sheet('KPCA(rbf)+LMNN')
for i in range(len(accu_score_1)):
    sheet.write(0,i,accu_score_1[i])
workbook.save('KPCA(rbf)+LMNN.xls') 